Creating Fact Schema for all of my fact tables --

In [ ]:
CREATE SCHEMA Fact;

Creating Dimenion Schema for all of my dimension tables --

In [ ]:
CREATE SCHEMA Dimension;

Verifying that both Schemas from above were successfully created --

In [ ]:
SELECT * FROM sys.schemas;

-- Creating Date Dimension Table --
CREATE TABLE Dimension.Date (
    DateKey int NOT NULL,
    FullDate date NOT NULL,
    DayName nvarchar(9) NOT NULL,
    DayNumber tinyint NOT NULL,
    MonthName varchar(9) NOT NULL,
    MonthNumber tinyint NOT NULL,
    Year tinyint NOT NULL

    CONSTRAINT PK_Date PRIMARY KEY CLUSTERED (
        DateKey ASC
    )
);

Creating Customer Dimension Table --

In [ ]:
CREATE TABLE Dimension.Customer (
    CustomerKey int IDENTITY (1,1) NOT NULL,
    CustomerAlternateKey int NOT NULL,
    FirstName nvarchar(50) NOT NULL,
    LastName nvarchar(50) NOT NULL,
    FullName nvarchar(100) NOT NULL,
    City nvarchar(50) NOT NULL,
    State char(2) NOT NULL,
    StateName nvarchar(20) NOT NULL,
    Zip char(5) NOT NULL

    CONSTRAINT PK_Customer PRIMARY KEY CLUSTERED (
        CustomerKey ASC
    )
);

Creating an Accounts Fact Table --

In [ ]:
CREATE TABLE Fact.Accounts (
    AccountID int IDENTITY (1,1) NOT NULL,
    DateKey int NOT NULL,
    CustomerKey int NOT NULL,
    InvoiceTotal money NOT NULL

    CONSTRAINT PK_Account PRIMARY KEY CLUSTERED (
        AccountID ASC
    )
);

Adding the relationship between Accounts Fact Table and Date Dimension Table --

In [ ]:
ALTER TABLE Fact.Accounts WITH CHECK ADD CONSTRAINT FK_Fact_Accounts_DakeKey_Dimension_Date FOREIGN KEY(DateKey)
REFERENCES Dimension.Date (DateKey);

In [ ]:
ALTER TABLE Fact.Accounts CHECK CONSTRAINT FK_Fact_Accounts_DakeKey_Dimension_Date;

Adding the relationship between Accounts Fact Table and Customer Dimension Table --

In [ ]:
ALTER TABLE Fact.Accounts WITH CHECK ADD CONSTRAINT FK_Fact_Accounts_CustomerKey_Dimension_Customer FOREIGN KEY(CustomerKey)
REFERENCES Dimension.Customer (CustomerKey);

In [ ]:
ALTER TABLE Fact.Accounts CHECK CONSTRAINT FK_Fact_Accounts_CustomerKey_Dimension_Customer;

Creating a View --

In [ ]:
CREATE VIEW dbo.AccountsByDate
    WITH SCHEMABINDING
    AS
        SELECT Accounts.AccountID, Accounts.CustomerKey, Accounts.InvoiceTotal,
            Date.FullDate, Date.DayName, Date.DayNumber, Date.MonthName,
            Date.MonthNumber, Date.Year
        FROM Fact.Accounts JOIN Dimension.[Date] ON Accounts.DateKey = Date.DateKey
;

In [ ]:
CREATE UNIQUE CLUSTERED INDEX IDX_AccountsByDate
    ON dbo.AccountsByDate (AccountID);

In [ ]:
SELECT * FROM dbo.AccountsByDate;

Creating a Columnstore Index for Performance Optimization of analytical queries --

In [ ]:
CREATE COLUMNSTORE INDEX IX_CS_FactAccounts
ON Fact.Accounts (AccountID, DateKey, CustomerKey, InvoiceTotal);

Rebuild Statement if fragmentation starts to happen (drops the index and rebuilds it entirely) --

ALTER INDEXIX_CS_FactAccounts

ON Fact.Accounts

REBUILD WITH (ONLINE = ON);